In [ ]:
def install_dhcp():
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh.connect(ip, username=user, password=pw, port=22)

    def run(cmd, show=True):
        """원격 명령 실행 + 실시간 출력"""
        stdin, stdout, stderr = ssh.exec_command(cmd)
        for line in iter(stdout.readline, ""):
            if show:
                print(line, end="")
                sys.stdout.flush()
        err = stderr.read().decode()
        if err and show:
            print("[ERROR]", err)
        return err

    print("DHCP 서버 설치를 시작합니다...")

    # 1. root 권한 확인
    err = run("if [[ $EUID -ne 0 ]]; then echo 'root 권한으로 실행해야 합니다.'; exit 1; fi")
    if err:
        ssh.close()
        return

    # 2. DHCP 서버 설치
    run("apt update -y")
    run("apt install -y isc-dhcp-server")

    # 3. 설정 파일 백업
    run('if [[ -f /etc/dhcp/dhcpd.conf ]]; then cp /etc/dhcp/dhcpd.conf /etc/dhcp/dhcpd.conf.bak_$(date +%F_%H%M%S); echo "기존 dhcpd.conf 백업 완료"; fi')

    # 4. 기본 DHCP 설정 작성
    dhcp_conf = r"""
default-lease-time 600;
max-lease-time 7200;
authoritative;

subnet 172.16.0.0 netmask 255.255.0.0 {
    range 172.16.254.6 172.16.254.200;
    option routers 172.16.0.1;
    option subnet-mask 255.255.0.0;
    option domain-name-servers 8.8.8.8, 8.8.4.4;
}
"""
    # 원격 파일 쓰기
    sftp = ssh.open_sftp()
    with sftp.file("/etc/dhcp/dhcpd.conf", 'w') as f:
        f.write(dhcp_conf)
    sftp.close()

    # 5. 인터페이스 지정
    run("sed -i 's/^INTERFACESv4=.*/INTERFACESv4=\"ens33\"/' /etc/default/isc-dhcp-server")


    # 6. 서비스 활성화 및 재시작
    run("systemctl enable isc-dhcp-server")
    run("systemctl restart isc-dhcp-server")

    # 7. 상태 출력
    print("==== DHCP 서비스 상태 ====")
    run("systemctl status isc-dhcp-server --no-pager")

    print("DHCP 서버 설치 및 기본 설정 완료!")